In [32]:
!pip install torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 1.3 MB/s eta 0:00:0000:0100:010m


In [3]:
import os
import torch
from torch.utils.data import Dataset
from torchvision.io import read_image
from torch import nn

In [4]:
path = "training"
labels = os.listdir(path)

def createDatasetIndex(path):
    result = {}
    for label in labels:
        if os.path.isfile(label):
            continue
        files = os.listdir(os.path.join(path,label))
        for file in files:
            result[len(result)] = (os.path.join(path,label,file),int(label))
    return result


In [5]:
class MNIST(Dataset):
    def __init__(self,path):
        self.items = createDatasetIndex(path)

    def __len__(self):
        return len(self.items)

    def __getitem__(self,index):
        img = read_image(self.items[index][0]) / 255.0
        label = self.items[index][1]
        return img,label


In [6]:
train_data = MNIST("training")
test_data = MNIST("testing")

In [7]:
from torch.utils.data import DataLoader

In [8]:
train_loader = DataLoader(train_data,batch_size=64,shuffle=True)
test_loader = DataLoader(test_data,batch_size=64,shuffle=True)

In [9]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.net = torch.nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5,padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(in_channels=6,out_channels=16,kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Flatten(),
            nn.Linear(in_features=400,out_features=120),
            nn.ReLU(),
            nn.Linear(in_features=120,out_features=84),
            nn.ReLU(),
            nn.Linear(in_features=84,out_features=10),
        )

    def forward(self,x):
        return self.net(x)

model = NeuralNetwork()

In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [11]:
def train(data_loader):
    size = len(data_loader.dataset)
    model.train()
    for batch,(X,y) in enumerate(data_loader):
        pred = model.forward(X)
        loss = loss_fn(pred,y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss,current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [12]:
def test(data_loader):
    size = len(data_loader.dataset)
    num_batches = len(data_loader)
    test_loss,correct = 0,0
    model.eval()
    with torch.no_grad():
        for X,y in data_loader:
            pred = model.forward(X)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test error: \n Accuracy: {(100*correct):>0.1f}%, Avg. loss: {test_loss:>8f} \n")
    return correct

In [13]:
for epoch in range(1,10):
    train(train_loader)
    correct = test(test_loader)
    if correct > 0.98:
        break

loss: 2.319910  [   64/60000]
loss: 0.312807  [ 6464/60000]
loss: 0.419672  [12864/60000]
loss: 0.333957  [19264/60000]
loss: 0.111833  [25664/60000]
loss: 0.186219  [32064/60000]
loss: 0.119482  [38464/60000]
loss: 0.144648  [44864/60000]
loss: 0.163873  [51264/60000]
loss: 0.047154  [57664/60000]
Test error: 
 Accuracy: 97.5%, Avg. loss: 0.080831 

loss: 0.070730  [   64/60000]
loss: 0.087217  [ 6464/60000]
loss: 0.027546  [12864/60000]
loss: 0.025250  [19264/60000]
loss: 0.222738  [25664/60000]
loss: 0.100965  [32064/60000]
loss: 0.057617  [38464/60000]
loss: 0.025117  [44864/60000]
loss: 0.015224  [51264/60000]
loss: 0.084889  [57664/60000]
Test error: 
 Accuracy: 98.1%, Avg. loss: 0.057562 



In [398]:
!pip install onnx onnxruntime;

In [14]:
import torch.onnx

In [16]:
pred = model(torch.unsqueeze(test_data[6000][0],0)).argmax(1)
print(int(pred),test_data[6000][1])

8 8


In [18]:
dummy_input = torch.randn(1,1,28,28)
torch.onnx.export(model,
      dummy_input,
      "mnist.onnx",
      export_params=True,
      opset_version=10,
      do_constant_folding=True,
      input_names = ['input'],
      output_names = ['output'],
      dynamic_axes={'input' : {0 : 'batch_size'},
                    'output' : {0 : 'batch_size'}}
)

============= Diagnostic Run torch.onnx.export version 2.0.0+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

